In [1]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re
from faker import Faker

2025-08-02 19:25:26,952 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-08-02 19:25:26,957 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-08-02 19:25:26,958 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [8]:
DB_NAME = "crm_outlook"

In [9]:
engine = create_pg_engine(db_name=DB_NAME)

In [10]:
# emails between jane and her clients
df = pd.read_sql("SELECT * FROM email_data", con=engine)
df.head()

,index,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,0.0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Introduction - Bankwell Financial Services for...,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and..."
1,1.0,2018-07-16 11:05:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services...,"Hi Jane,\n\nThank you for reaching out. AbbVie..."
2,2.0,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Re: Introduction - Bankwell Financial Services...,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfec..."
3,3.0,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking wit..."
4,4.0,2018-08-02 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The ..."


## Look for emails

In [9]:
df

,index,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,0.0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Introduction - Bankwell Financial Services for...,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and..."
1,1.0,2018-07-16 11:05:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services...,"Hi Jane,\n\nThank you for reaching out. AbbVie..."
2,2.0,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Re: Introduction - Bankwell Financial Services...,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfec..."
3,3.0,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking wit..."
4,4.0,2018-08-02 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The ..."
...,...,...,...,...,...,...,...,...
811,255.0,2025-07-08 09:40:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Thanks Jane,\n\nThank you for the update. Seem..."
812,257.0,2025-07-09 16:10:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Hi jane, Yes, this needs further thought. I c..."
813,1.0,2025-05-21 09:47:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - ESG Ratings Dashboard Review,"Hi Jane, Thank you for bringing this to my att..."
814,NaN,2025-05-21 06:15:00,Adam Clay,adam.clay@compass.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass - ESG Ratings Dashboard Review,"Hi Adam, Hope you are doing well. I wanted to ..."


In [21]:
to_adam_email_df = df[df['to_emails'].str.contains('adam.clay@compass.com', na=False)][
    ['email_body', 'email_subject', 'email_timestamp']
]

display(to_adam_email_df)


,email_body,email_subject,email_timestamp
12,"Hi Adam,\n\nHope you're having a good week. I ...",Meeting Request: Compass Expansion Financing,2018-09-18 09:00:00
14,"Hi Adam,\n\nExcellent. Tuesday at 10 AM ET is ...",Re: Meeting Request: Compass Expansion Financing,2018-09-18 13:30:00
61,"Hi Adam,\n\nFollowing our discussions and revi...",Compass - Credit Facility Term Sheet,2019-09-10 14:45:00
102,"Hi Adam,\n\nFollowing up on our call last week...",Compass - Credit Facility Next Steps,2020-09-17 15:00:00
103,"Hi Adam,\n\nExcellent news. I'll instruct our ...",Re: Compass - Credit Facility Next Steps,2020-09-21 11:45:00
150,"Hi Adam,\n\nHope the documentation review for ...",Compass - Credit Facility Documentation Status,2021-09-28 15:30:00
151,"Hi Adam,\n\nSounds good. We'll prepare for a p...",Re: Compass - Credit Facility Documentation St...,2021-09-29 10:15:00
195,"Hi Adam,\n\nGreat news - confirming that the $...",Compass - Credit Facility Closed,2022-10-06 10:00:00
239,"Hi Adam,\n\nHope you're well. Just checking in...",Compass - Utilizing the Credit Facility,2023-09-29 14:00:00
273,"Hi Adam,\n\nHope you're having a good week. I ...",Meeting Request: Compass Expansion Financing,2018-09-18 09:00:00


In [29]:
adam_email_df = df[df['from_email'].str.contains('adam.clay@compass.com', na=False)][
    ['email_body', 'email_subject', 'email_timestamp']
]

with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(adam_email_df)

,email_body,email_subject,email_timestamp
13,"Hi Jane,\n\nGood timing. Yes, we are evaluating options for a new credit facility. I can chat next Tuesday morning. Does 10 AM ET work?\n\nRegards,\n\n--\nAdam Clay\nVP\nCompass\nEmail: adam.clay@compass.com\nPhone: -2931\n",Re: Meeting Request: Compass Expansion Financing,2018-09-18 13:15:00
274,"Hi Jane,\n\nGood timing. Yes, we are evaluating options for a new credit facility. I can chat next Tuesday morning. Does 10 AM ET work?\n\nRegards,\n\n--\nAdam Clay\nVP\nCompass\nEmail: adam.clay@compass.com\nPhone: -2931\n",Re: Meeting Request: Compass Expansion Financing,2018-09-18 13:15:00
813,"Hi Jane, Thank you for bringing this to my attention. I know our board is particularly concerned with the Carbon Score index on the dashboard. Which stocks have been added to the CSI index that were not there previously? Thanks, Adam",Compass - ESG Ratings Dashboard Review,2025-05-21 09:47:00
815,"Jane, I saw an update in regulations from the SEC on AML. Compass legal team has some updated policies on funds documentation to comply that I would like to make sure we are using going forward. Have you heard of any other AML regulation updates we should be aware of? Thanks, Adam",Compass - AML Requirements & Policy Developments,2025-06-14 18:15:00


In [11]:
jessica_email_df = df[df['from_email'].str.contains('jessica.palmer@hasbro.com', na=False)][
    ['email_body', 'email_subject', 'email_timestamp']
]

with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(jessica_email_df)


,email_body,email_subject,email_timestamp
22,"Hi Jane,\n\nThanks for the information. FX volatility is certainly a key concern for us, especially with EUR and GBP. I'd like to understand the costs and flexibility associated with forward contracts versus options. Could we schedule time to discuss this, perhaps mid-November?\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - FX Risk Management for International Sales,2018-11-02 14:00:00
68,"Hi Jane,\n\nThis summary is exactly what we needed. Thank you. We'll review this internally and aim to decide on our preferred strategy for H1 2020 hedging shortly. I'll let you know once we're ready to proceed.\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - FX Hedging Strategy - Forwards vs. Options,2019-11-20 10:30:00
110,"Hi Jane,\n\nThanks for the nudge. We've decided to primarily use forward contracts for the bulk of our H1 2021 EUR and GBP exposure to lock in budget rates, perhaps with a small layer of options for flexibility if pricing is attractive.\n\nCan you provide indicative pricing for 3-month and 6-month forwards based on current markets, and maybe some indicative costs for simple 6-month call options (say, 2-3% out-of-the-money)?\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - Decision on H1 2021 FX Hedging,2020-11-25 10:10:00
159,"Hi Jane,\n\nExcellent, thanks for the prompt execution and confirmation. All looks good. Appreciate getting these locked in before year-end.\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - Execution of H1 2022 FX Hedges,2021-12-02 09:45:00
202,"Hi Jane,\n\nThanks for sending the performance review. It looks like the forwards provided good budget certainty in a volatile period. Let's schedule time in early January to discuss the H1 2023 strategy. Can you propose some times?\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - Review of H1 2022 FX Hedge Performance,2022-12-07 11:00:00
246,"Hi Jane,\n\nYes, let's connect. How about December 11th at 11 AM ET?\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - Strategy Discussion for H1 2024 FX Hedging,2023-11-22 10:30:00
256,"Thanks Jane,\n\nI will follow up if I have any issues. Happy holidays! \n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - Strategy Discussion for H1 2024 FX Hedging,2023-12-12 11:30:00
259,"Dear Jane,\n\nThank you for your email. Jane,\n\nAny updates on AML requirements?\n\nAppreciate it,\nJessica Palmer\n\nBest regards,\nJessica Palmer",Re: Hasbro - Disclosure Timeline Revision Review,2025-04-30 00:00:00
284,"Hi Jane,\n\nThanks for the information. FX volatility is certainly a key concern for us, especially with EUR and GBP. I'd like to understand the costs and flexibility associated with forward contracts versus options. Could we schedule time to discuss this, perhaps mid-November?\n\nRegards,\n\n--\nJessica Palmer\nVP\nHasbro\nEmail: jessica.palmer@hasbro.com\nPhone: 909.878.8329x31984\n",Re: Hasbro - FX Risk Management for International Sales,2018-11-02 14:00:00
318,"Dear Jane,\n\nThank you for your email. Jane,\n\nAny updates on AML requirements?\n\nAppreciate it,\nJessica Palmer\n\nBest regards,\nJessica Palmer",Re: Hasbro - Disclosure Timeline Revision Review,2025-04-30 00:00:00


In [32]:
df['from_email'].unique()

array(['jane.moneypenny@bankwell.com', 'cynthia.hobbs@abbvie.com',
       'jennifer.phelps@aerovironment.com', 'kyle.waters@amedisys.com',
       'denise.moore@celestica.com', 'adam.clay@compass.com',
       'amy.winters@gamestopcorp.com',
       'roberto.martin@guardanthealth.com', 'jessica.palmer@hasbro.com',
       'timothy.ochoa@hyatthotels.com',
       'michelle.jenkins@intuitivesurgical.com',
       'ronnie.gray@laddercapitalcorp.com',
       'lisa.kennedy@lockheedmartincorporation.com',
       'david.moreno@manpowergroup.com', 'mary.vasquez@marriott.com',
       'anna.lawrence@matson.com', 'tracey.smith@medtronic.com',
       'kelly.smith@presidiopropertytrust.com',
       'ryan.holloway@bankwell.com'], dtype=object)

In [48]:
cynthia_email_df = df[df['from_email'].str.contains('cynthia.hobbs@abbvie.com', na=False)][
    ['email_body', 'email_subject', 'email_timestamp']
]


with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(cynthia_email_df)

,email_body,email_subject,email_timestamp
1,"Hi Jane,\n\nThank you for reaching out. AbbVie is always interested in optimizing its financial operations. I'd be happy to schedule a brief call. How does Tuesday at 2 PM CT sound?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: Introduction - Bankwell Financial Services for AbbVie,2018-07-16 11:05:00
54,"Hi Jane,\n\nThanks for checking in. So far, everything looks good from our end. The initial sweeps occurred as expected this morning. We'll monitor it closely over the next few days, but the transition seems smooth. Appreciate your team's support.\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Cash Management Implementation Update,2019-06-13 10:10:00
91,"Hi Jane,\n\nA quarterly review sounds good. How about July 7th at 11 AM CT?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Quarterly Relationship Review,2020-06-16 13:00:00
140,"Hi Jane,\n\nYes, potentially. We have a number of recurring payments to suppliers in Canada and some EU countries where IAT might be applicable. Please send over some information on the service and typical cost savings compared to wires.\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Information on International ACH (IAT) Payments,2021-06-11 09:45:00
185,"Hi Jane,\n\nYes, we'd like to start using IAT for our Canadian dollar payments initially. A call with your operations team to understand the setup and required beneficiary information would be helpful. Can they propose some times next week?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Setting up IAT Payments,2022-06-23 10:00:00
230,"Hi Jane,\n\nYes, let's schedule the Q3 review. How about July 25th at 1 PM CT?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Quarterly Relationship Review - Q3 2023,2023-07-07 13:30:00
258,"Hi Jane,\n\nThank you for reaching out. AbbVie is always interested in optimizing its financial operations. I'd be happy to schedule a brief call. How does Tuesday at 2 PM CT sound?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: Introduction - Bankwell Financial Services for AbbVie,2018-07-16 11:05:00
314,"Hi Jane,\n\nThanks for checking in. So far, everything looks good from our end. The initial sweeps occurred as expected this morning. We'll monitor it closely over the next few days, but the transition seems smooth. Appreciate your team's support.\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Cash Management Implementation Update,2019-06-13 10:10:00
354,"Hi Jane,\n\nA quarterly review sounds good. How about July 7th at 11 AM CT?\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Quarterly Relationship Review,2020-06-16 13:00:00
400,"Hi Jane,\n\nYes, potentially. We have a number of recurring payments to suppliers in Canada and some EU countries where IAT might be applicable. Please send over some information on the service and typical cost savings compared to wires.\n\nBest,\n\n--\nCynthia Hobbs\nDirector\nAbbVie\nEmail: cynthia.hobbs@abbvie.com\nPhone: (446)673-8121x90878\n",Re: AbbVie - Information on International ACH (IAT) Payments,2021-06-11 09:45:00


In [47]:
denise_email_df = df[df['from_email'].str.contains('denise.moore@celestica.com', na=False)][
    ['email_body', 'email_subject', 'email_timestamp']
]


with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(denise_email_df)

,email_body,email_subject,email_timestamp
10,"Hi Jane,\n\nThis comparison is very useful, thank you. The enhanced yield MMF looks interesting, could you provide the specific fund prospectus and details on the underlying assets? We need to ensure it fits within our investment policy statement.\n\nThanks,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Short-Term Investment Options,2018-09-06 10:30:00
60,"Hi Jane,\n\nThanks for the confirmation. Appreciate the smooth handling of the investment.\n\nBest,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - MMF Investment Confirmation,2019-08-14 11:30:00
100,"Hi Jane,\n\nA mid-year check-in sounds good. How about September 8th at 3 PM ET?\n\nThanks,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Mid-Year Investment Portfolio Check-in,2020-08-21 10:00:00
149,"Hi Jane,\n\nThanks for sending the report over promptly. Appreciate the overview and ongoing monitoring.\n\nBest,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Investment Portfolio Performance Q3 YTD,2021-09-09 11:15:00
193,"Hi Jane,\n\nThat's a timely point. Our liquidity position is strong currently. Let's schedule a call next week to look at options for potentially extending duration slightly to capture higher yields. How about Tuesday at 2 PM ET?\n\nThanks,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Rising Rate Environment & Investment Strategy,2022-09-15 10:00:00
238,"Hi Jane,\n\nThanks for confirming the CD purchase. Appreciate locking in that rate.\n\nBest,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Investment Execution: 6-Month CD,2023-09-12 11:00:00
271,"Hi Jane,\n\nThis comparison is very useful, thank you. The enhanced yield MMF looks interesting, could you provide the specific fund prospectus and details on the underlying assets? We need to ensure it fits within our investment policy statement.\n\nThanks,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Short-Term Investment Options,2018-09-06 10:30:00
323,"Hi Jane,\n\nThanks for the confirmation. Appreciate the smooth handling of the investment.\n\nBest,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - MMF Investment Confirmation,2019-08-14 11:30:00
363,"Hi Jane,\n\nA mid-year check-in sounds good. How about September 8th at 3 PM ET?\n\nThanks,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Mid-Year Investment Portfolio Check-in,2020-08-21 10:00:00
409,"Hi Jane,\n\nThanks for sending the report over promptly. Appreciate the overview and ongoing monitoring.\n\nBest,\n\n--\nDenise Moore\nSenior Director\nCelestica\nEmail: denise.moore@celestica.com\nPhone: 754-579-1511x763\n",Re: Celestica - Investment Portfolio Performance Q3 YTD,2021-09-09 11:15:00


## Check Jane's Preference

In [38]:
pd.read_sql("SELECT * FROM preferences where email = 'jane.moneypenny@bankwell.com' ", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,full,full,full,full
1,2023-04-02 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,full,short
2,2023-07-26 14:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,none,none,full,short
3,2023-11-04 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,none,none
4,2024-01-30 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,short,full,none,full
5,2024-03-10 09:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,full,none,full,full


## Check Meeting Data

In [40]:
df1 = pd.read_sql("SELECT * FROM meeting_data", con=engine)
df1.head()

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,0.0,2018-07-17 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services...
1,1.0,2018-07-24 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financia...
2,2.0,2018-08-07 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,3.0,2018-08-15 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,4.0,2018-08-29 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Ma...


# Potential Init_state selection

### 1) Cynthis Hobbs

In [42]:
cynthia_meeting_df = df1[df1['invitee_emails'].str.contains('cynthia.hobbs@abbvie.com', na=False)][
    ['invitees', 'meeting_subject', 'meeting_timestamp']
]
cynthia_meeting_df

,invitees,meeting_subject,meeting_timestamp
0,Cynthia Hobbs,AbbVie - Introductory Call & Bankwell Services...,2018-07-17 15:00:00
29,Cynthia Hobbs,AbbVie - Q2 2020 Relationship Review,2020-07-07 12:00:00
45,Cynthia Hobbs,AbbVie - IAT Payment Setup Process Review,2021-06-29 10:00:00
74,Cynthia Hobbs,AbbVie - Q3 2023 Relationship Review,2023-07-25 14:00:00
84,Cynthia Hobbs,AbbVie - Q4 2023 Relationship Review,2024-01-25 14:00:00
94,Cynthia Hobbs,AbbVie - Introductory Call & Bankwell Services...,2018-07-17 15:00:00
123,Cynthia Hobbs,AbbVie - Q2 2020 Relationship Review,2020-07-07 12:00:00
139,Cynthia Hobbs,AbbVie - IAT Payment Setup Process Review,2021-06-29 10:00:00
168,Cynthia Hobbs,AbbVie - Q3 2023 Relationship Review,2023-07-25 14:00:00
178,Cynthia Hobbs,AbbVie - Q4 2023 Relationship Review,2024-01-25 14:00:00


Good meetings to select:
1. Cynthia Hobbs	AbbVie - ESG Fund Portfolio & Benchmark Review	2025-05-03 10:00:00
2. Cynthia Hobbs    AbbVie - Growth & Fixed Income Launch Overview	2025-05-24 15:00:00
3. Cynthia Hobbs	AbbVie - Annual Plan & Strategic Priorities	2025-06-13 16:00:00


### 2) Denise Moore

In [45]:
dennis_meeting_df = df1[df1['invitee_emails'].str.contains('denise.moore@celestica.com', na=False)][
    ['invitees', 'meeting_subject', 'meeting_timestamp']
]
with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(dennis_meeting_df)

,invitees,meeting_subject,meeting_timestamp
4,Denise Moore,Celestica - Initial Discussion on Liquidity Management,2018-08-29 14:00:00
33,Denise Moore,Celestica - Mid-Year Investment Portfolio Check-in,2020-09-08 15:00:00
50,Denise Moore,Celestica - Discuss Investment Strategy (Rising Rates),2021-09-21 14:00:00
87,Denise Moore,Celestica - Discuss CD Maturity and Reinvestment Options,2024-02-13 14:00:00
98,Denise Moore,Celestica - Initial Discussion on Liquidity Management,2018-08-29 14:00:00
127,Denise Moore,Celestica - Mid-Year Investment Portfolio Check-in,2020-09-08 15:00:00
144,Denise Moore,Celestica - Discuss Investment Strategy (Rising Rates),2021-09-21 14:00:00
181,Denise Moore,Celestica - Discuss CD Maturity and Reinvestment Options,2024-02-13 14:00:00
231,Denise Moore,Celestica - Carbon Score & ESG Exclusion Analysis,2025-05-18 10:00:00
232,Denise Moore,Celestica - Carbon Score & ESG Exclusion Analysis,2025-05-18 10:00:00


Good meeting to select for 

1. Denise Moore Celestica - Mid-Year Objective Adjustment Review	2025-06-22 16:00:00 - Finance
2. Denise Moore	Celestica - Denise Moore	Celestica - Cross-Border Regulation & Jurisdiction Briefing	2025-05-25 10:00:00 - Legal
3. Denise Moore	Celestica - Carbon Score & ESG Exclusion Analysis	2025-05-18 10:00:00 - ESG

Idea: Between report 1 & 2 can show how the meeting categorizer difference

### 3) Jessica Palmer

In [46]:
jessica_meeting_df = df1[df1['invitee_emails'].str.contains('jessica.palmer@hasbro.com', na=False)][
    ['invitees', 'meeting_subject', 'meeting_timestamp']
]
with pd.option_context('display.max_rows', None,        
                       'display.max_columns', None,     
                       'display.max_colwidth', None,    
                       'display.expand_frame_repr', False):  
    display(jessica_meeting_df)

,invitees,meeting_subject,meeting_timestamp
8,Jessica Palmer,Hasbro - Discuss FX Risk Management Strategies (Forwards vs. Options),2018-11-14 11:00:00
67,Jessica Palmer,Hasbro - Discuss H1 2023 FX Hedging Strategy,2023-01-10 10:00:00
81,Jessica Palmer,Hasbro - Finalize H1 2024 FX Hedging Strategy,2023-12-11 11:00:00
102,Jessica Palmer,Hasbro - Discuss FX Risk Management Strategies (Forwards vs. Options),2018-11-14 11:00:00
161,Jessica Palmer,Hasbro - Discuss H1 2023 FX Hedging Strategy,2023-01-10 10:00:00
175,Jessica Palmer,Hasbro - Finalize H1 2024 FX Hedging Strategy,2023-12-11 11:00:00
188,Jessica Palmer,Hasbro - Discuss Portfolio Diversification and Bond Strategy,2024-02-04 14:30:00
189,Jessica Palmer,Hasbro - Strategic Targets Rollout Discussion,2025-05-01 11:00:00
190,Jessica Palmer,Hasbro - Strategic Targets Rollout Discussion,2025-05-01 11:00:00
191,Jessica Palmer,Hasbro - Disclosure Timeline Revision Review,2025-05-01 12:00:00


Good meetings to select:
1. Jessica Palmer	Hasbro - Discuss Portfolio Diversification and Risk Strategy	2025-07-10 14:30:00 - Risk
2. Jessica Palmer	Hasbro - ESG Integration Framework & Screening	2025-05-17 14:00:00 ESG
3. Jessica Palmer	Hasbro - Strategic Targets Rollout Discussion	2025-05-01 11:00:00 - General Finance

# Report Testing

In [1]:
from currensee.agents.complete_graph import compiled_graph
from currensee.agents.tools.finance_tools import (generate_macro_table,
    retrieve_client_industry_news,
    retrieve_holdings_news,
    retrieve_macro_news,
    summarize_finance_outputs)
    
import joblib
import warnings
warnings.filterwarnings("ignore")
from currensee.utils.output_utils_dynamic import (
    generate_report,
    format_news_summary_to_html,
    format_paragraph_summary_to_html,
    save_html_to_file,
    convert_html_to_pdf,
)


warnings.filterwarnings("ignore")


2025-08-03 03:43:26,556 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-08-03 03:43:26,561 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-08-03 03:43:26,563 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
from datetime import date

today = date.today()
today_date = today.strftime("%Y%m%d")
print(today_date)

20250803


In [ ]:
# Test 1 - Jessica Palmer	Hasbro - Discuss Portfolio Diversification and Risk Strategy	2025-07-10 14:30:00 - Risk

In [3]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Jessica Palmer",
    "client_email": "jessica.palmer@hasbro.com",
    "meeting_timestamp": "2025-07-10 14:30:00",
    "meeting_description": "Hasbro - Discuss Portfolio Diversification and Risk Strategy",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2025-05-17 to 2025-07-10
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 6 macro articles remain
DEBUG: Filtering date range is from 2025-05-17 to 2025-07-10
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 15 articles remain
DEBUG: Filtering date range is from 2025-05-17 to 2025-07-10
DEBUG: Processing holding 'Walmart Inc'
DEBUG: Executing 6 site-specific queries for holding 'Walmart Inc'
DEBUG: Aggregated 24 unique results for holding 'Walmart Inc'
DEBUG: After filtering: 13 articles for holding 'Walmart Inc'
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 13 articles for holding 'Amazon.com Inc'
DEBUG: Processing hold

In [4]:
jessica_07_10 = generate_report(result)
save_html_to_file(jessica_07_10, 'jessica_07_10.html')

In [ ]:
# Test 2 - Jessica - Jessica Palmer	Hasbro - ESG Integration Framework & Screening	2025-05-17 14:00:00 ESG

In [5]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Jessica Palmer",
    "client_email": "jessica.palmer@hasbro.com",
    "meeting_timestamp": "2025-05-17 14:00:00",
    "meeting_description": "ESG Integration Framework & Screening",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2025-05-01 to 2025-05-17
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 7 macro articles remain
DEBUG: Filtering date range is from 2025-05-01 to 2025-05-17
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 11 articles remain
DEBUG: Filtering date range is from 2025-05-01 to 2025-05-17
DEBUG: Processing holding 'Walmart Inc'
DEBUG: Executing 6 site-specific queries for holding 'Walmart Inc'
DEBUG: Aggregated 24 unique results for holding 'Walmart Inc'
DEBUG: After filtering: 13 articles for holding 'Walmart Inc'
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing hold

In [6]:
jessica_05_17 = generate_report(result)
save_html_to_file(jessica_05_17, 'jessica_05_17.html')

In [ ]:
#Jessica 3 - 3. Jessica Palmer	Hasbro - Strategic Targets Rollout Discussion	2025-05-01 11:00:00 - General Finance

In [7]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Jessica Palmer",
    "client_email": "jessica.palmer@hasbro.com",
    "meeting_timestamp": "2025-05-01 11:00:00",
    "meeting_description": "Hasbro - Strategic Targets Rollout Discussion",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2025-03-29 to 2025-05-01
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 14 macro articles remain
DEBUG: Filtering date range is from 2025-03-29 to 2025-05-01
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 12 articles remain
DEBUG: Filtering date range is from 2025-03-29 to 2025-05-01
DEBUG: Processing holding 'Walmart Inc'
DEBUG: Executing 6 site-specific queries for holding 'Walmart Inc'
DEBUG: Aggregated 24 unique results for holding 'Walmart Inc'
DEBUG: After filtering: 13 articles for holding 'Walmart Inc'
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing hol

In [8]:
jessica_05_01 = generate_report(result)
save_html_to_file(jessica_05_01, 'jessica_05_01.html')

In [ ]:
#Dennis 1 - Denise Moore Celestica - Mid-Year Objective Adjustment Review 2025-06-22 16:00:00 - Finance

In [11]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Denise Moore",
    "client_email": "denise.moore@celestica.com",
    "meeting_timestamp": "2025-06-22 16:00:00",
    "meeting_description": "Celestica - Mid-Year Objective Adjustment Review",
}

result = compiled_graph.invoke(init_state)


DEBUG: Filtering date range is from 2025-06-15 to 2025-06-22
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 11 macro articles remain
DEBUG: Filtering date range is from 2025-06-15 to 2025-06-22
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 9 articles remain
DEBUG: Filtering date range is from 2025-06-15 to 2025-06-22
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 13 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platforms

In [12]:
denise_06_22 = generate_report(result)
save_html_to_file(denise_06_22, 'denise_06_22.html')

In [ ]:
#Denise 2 - Denise Moore Celestica - Denise Moore Celestica - Cross-Border Regulation & Jurisdiction Briefing 2025-05-25 10:00:00 - Legal

In [13]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Denise Moore",
    "client_email": "denise.moore@celestica.com",
    "meeting_timestamp": "2025-05-25 10:00:00",
    "meeting_description": "Celestica - Cross-Border Regulation & Jurisdiction Briefing",
}

result = compiled_graph.invoke(init_state)


DEBUG: Filtering date range is from 2025-05-21 to 2025-05-25
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 11 macro articles remain
DEBUG: Filtering date range is from 2025-05-21 to 2025-05-25
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 10 articles remain
DEBUG: Filtering date range is from 2025-05-21 to 2025-05-25
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platform

In [14]:
denise_05_25 = generate_report(result)
save_html_to_file(denise_05_25, 'denise_05_25.html')

In [ ]:
#Denise 3 - Denise Moore Celestica - Carbon Score & ESG Exclusion Analysis 2025-05-18 10:00:00 - ESG

In [15]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Denise Moore",
    "client_email": "denise.moore@celestica.com",
    "meeting_timestamp": "2025-05-18 10:00:00",
    "meeting_description": "Celestica - Cross-Border Regulation & Jurisdiction Briefing",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2024-02-13 to 2025-05-18
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 11 macro articles remain
DEBUG: Filtering date range is from 2024-02-13 to 2025-05-18
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 9 articles remain
DEBUG: Filtering date range is from 2024-02-13 to 2025-05-18
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 13 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platforms

In [16]:
denise_05_18 = generate_report(result)
save_html_to_file(denise_05_18, 'denise_05_18.html')

In [ ]:
#Cynthis 1 - Cynthia Hobbs AbbVie - ESG Fund Portfolio & Benchmark Review 2025-05-03 10:00:00

In [17]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Cynthia Hobbs",
    "client_email": "cynthia.hobbs@abbvie.com",
    "meeting_timestamp": "2025-05-03 10:00:00",
    "meeting_description": "AbbVie - ESG Fund Portfolio & Benchmark Review",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2024-01-25 to 2025-05-03
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 11 macro articles remain
DEBUG: Filtering date range is from 2024-01-25 to 2025-05-03
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 13 articles remain
DEBUG: Filtering date range is from 2024-01-25 to 2025-05-03
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platform

In [18]:
cynthia_05_03 = generate_report(result)
save_html_to_file(cynthia_05_03, 'cynthia_05_03.html')

In [ ]:
#Cythina 2- Cynthia Hobbs AbbVie - Growth & Fixed Income Launch Overview 2025-05-24 15:00:00


In [19]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Cynthia Hobbs",
    "client_email": "cynthia.hobbs@abbvie.com",
    "meeting_timestamp": "2025-05-24 15:00:00",
    "meeting_description": "AbbVie - Growth & Fixed Income Launch Overview",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2025-05-18 to 2025-05-24
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 9 macro articles remain
DEBUG: Filtering date range is from 2025-05-18 to 2025-05-24
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 10 articles remain
DEBUG: Filtering date range is from 2025-05-18 to 2025-05-24
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platforms

In [21]:
cynthia_05_24 = generate_report(result)
save_html_to_file(cynthia_05_24, 'cynthia_05_24.html')

In [ ]:
#Cynthis 3 - Cynthia Hobbs AbbVie - Annual Plan & Strategic Priorities 2025-06-13 16:00:00

In [22]:
init_state = {
    "user_email" : "jane.moneypenny@bankwell.com",
    "client_name": "Cynthia Hobbs",
    "client_email": "cynthia.hobbs@abbvie.com",
    "meeting_timestamp": "2025-06-13 16:00:00",
    "meeting_description": "AbbVie - Annual Plan & Strategic Priorities",
}

result = compiled_graph.invoke(init_state)

DEBUG: Filtering date range is from 2025-06-07 to 2025-06-13
DEBUG: Executing 6 site-specific queries for macro news
DEBUG: Aggregated 36 unique macro results from all sites
DEBUG: After filtering: 12 macro articles remain
DEBUG: Filtering date range is from 2025-06-07 to 2025-06-13
DEBUG: Executing 6 site-specific queries for client industry news
DEBUG: Aggregated 48 unique results from all sites
DEBUG: After filtering: 10 articles remain
DEBUG: Filtering date range is from 2025-06-07 to 2025-06-13
DEBUG: Processing holding 'Amazon.com Inc'
DEBUG: Executing 6 site-specific queries for holding 'Amazon.com Inc'
DEBUG: Aggregated 24 unique results for holding 'Amazon.com Inc'
DEBUG: After filtering: 12 articles for holding 'Amazon.com Inc'
DEBUG: Processing holding 'Meta Platforms Inc'
DEBUG: Executing 6 site-specific queries for holding 'Meta Platforms Inc'
DEBUG: Aggregated 24 unique results for holding 'Meta Platforms Inc'
DEBUG: After filtering: 13 articles for holding 'Meta Platform

In [23]:
cynthia_06_13 = generate_report(result)
save_html_to_file(cynthia_06_13, 'cynthia_06_13.html')

In [ ]:
##Change Jessica's meeting date
#Add client box